LIS (longest increasing subsequence), LPS (longest palindromic sequence), LCSSeq (longest common subsequence), LCS (longest common substring), make\_amount, make\_changes, ...

Given a string, a partitioning of the string is a palindrome partitioning if every substring of the partition is a palindrome. For example, “aba|b|bbabb|a|b|aba” is a palindrome partitioning of “ababbbabbababa”. Determine the fewest cuts needed for palindrome partitioning of a given string. 


Given two strings of size m, n and operations of insert (I), delete (D), and replace (R)
all at equal cost. Find the minimum # of edits required to transform a string into another.

In [1]:
from functools import lru_cache

def recap(kv, k, v):
    return (kv[0] + k, kv[1] + v)

def knapsack01(skus, capacity): # a seq of costs & values.
    @lru_cache(maxsize=None) # by capacity c and n items.
    def prog(n, c): # returns a tuple of (a yield, items).
        if n == 0:
            return (0, ()) # a tuple of ($0, none items).
        sku = skus[n-1]
        cases = (recap(prog(n-1, c-cost), value, seq)
                 for (cost, value), seq in ((sku, (n-1,)), ((0, 0), ()))
                 if c >= cost) # at most two cases
        return max(cases, key=lambda kv: kv[0])
    return prog(len(skus), capacity)

def knapsack_unbounded(skus, capacity):
    @lru_cache(maxsize=None)
    def prog(c):
        cases = (recap(prog(c - sku[0]), sku[1], (i,))
                 for i, sku in enumerate(skus)
                 if c >= sku[0])
        return max(cases, key=lambda kv: kv[0], default=(0, ()))
    return prog(capacity)

skus = [(2, 2), (1, 1), (4, 10), (1, 2), (12, 4)]
assert (15, (0, 1, 2, 3)) == knapsack01(skus, 16)
assert (36, (3, 3, 3, 2, 2, 2)) == knapsack_unbounded(skus, 15)

In [11]:
from enum import Enum

Edit = Enum("Edit", {"I": "Insert", "D": "Delete", "R": "Replace", "M": "Match"})

def edit(a, b):
    @lru_cache(maxsize=None)
    def prog(i, o):
        if i == 0 or o == 0:
            return (i+o, ())
        m = int(a[i-1] != b[o-1]) # match costs 1 or 0.
        cases = (recap(prog(i-1, o-1), m, (Edit.M,)),
                 recap(prog(i-1, o), 1, (Edit.D,)),
                 recap(prog(i, o-1), 1, (Edit.I,)))
        return min(cases, key=lambda e: e[0])
    return prog(len(a), len(b))

assert (3, (Edit.M,) * 6 + (Edit.I,)) == edit('kitten', 'sitting')  # 6 matches and 1 insert.

In [3]:
def renumerate(a, stop=-1):
    return zip(range(len(a)+stop, stop, -1), reversed(a))

In [4]:
def maxima(iterable, key=lambda e: e):
    maxima = []
    for e in iterable:
        if not maxima or key(e) == key(maxima[0]):
            maxima.append(e)
        elif key(e) > key(maxima[0]):
            maxima = [e]
    return maxima

def lis(a):
    @lru_cache(maxsize=None)
    def lisuff(i):
        cases = (e + (a[i],)
                 for j in range(i)
                 for e in lisuff(j) 
                 if e[-1] <= a[i])  # 0 <= j < i
        return maxima(cases, key=len) or [(a[i],)]
    return maxima((e
                   for i in range(len(a))
                   for e in lisuff(i)), key=len)

assert [(1, 5, 6), (1, 2, 3)] == lis((7, 8, 1, 5, 6, 2, 3))

In [5]:
from itertools import chain

def lps(s): # longest palindromic subsequence http://goo.gl/nTiQvX
    @lru_cache(maxsize=None)
    def prog(b, e): # begin(b), end(e)
        if e - b == 1:
            return [s[b:e]]
        elif e - b == 2:
            if s[b] == s[b+1]:
                return [s[b:e]]
            else:
                return [s[b:b+1], s[b+1:b+2]]
        elif s[b] == s[e-1]:
            return [s[b]+ss+s[b] for ss in prog(b+1, e-1)]
        else:
            return maxima((chain(prog(b+1, e), prog(b, e-1))), key=len)
    return prog(0, len(s))
        
assert ['abzba'] == list(lps('xaybzba'))
assert ['bab', 'aba'] == list(lps('abab'))

In [6]:
def lcsseq(a, b):
    n, m = len(a), len(b)
    def prog(i, j, memos={}):
        args = (i, j)
        if args not in memos:
            if i == -1 or j == -1:
                m = ['']
            elif a[i] == b[j]:
                m = [a[i] + s for s in prog(i-1, j-1)]
            else:
                m = maxima(chain(prog(i-1, j), prog(i, j-1)), key=len)
            memos[args] = m
        return memos[args]
    return prog(n-1, m-1)

assert ['aba', 'bab'] == list(lcsseq('abab', 'baba'))
assert {'ecaba'} == set(lcsseq('apbcadcqer', 'rasbtaucve'))

In [7]:
def lcs(a, b):
    @lru_cache(maxsize=None)
    def lcsuff(i, j): # returns a tuple of (offset, count)
        if i == -1 or j == -1 or a[i] != b[j]:
            return (i, 0)
        o, c = lcsuff(i-1, j-1)
        return (o, c+1) if c > 0 else (i, 1)
    m, n = len(a), len(b)
    z = maxima((lcsuff(i, j) for i in range(m) for j in range(n)), key=lambda e: e[1])
    return [a[o:o+c] for o, c in z]

assert ['aba', 'bab'] == lcs('abab', 'baba')

In [8]:
def max_profit(a, k): # at most k transactions.
    n = len(a)
    @lru_cache(maxsize=None)
    def prog(start, k):
        if k == 0 or start > n-2:
            return (0, ())
        else:
            cases = ((a[s] - a[b] + prog(s+1, k-1)[0],
                      ((b, s),) + prog(s+1, k-1)[1])
                     for b in range(start, n-1)
                     for s in range(b+1, n)
                     if a[b] < a[s])
            return max(cases, key=lambda e: e[0])
    return prog(0, k)

a = [1, 9, 2, 8, 3, 7, 4, 5, 6]
assert (18, ((0, 1), (2, 3), (4, 5))) == max_profit(a, 3)

In [9]:
def recalc(kv, k, v):
    return (max(k, kv[0]), v + kv[1])

def bookshelf(books, p): # at most p partitions.
    n = len(books)
    @lru_cache(maxsize=None)
    def prog(p, s): # partitions (p), range start (s).
        if p == 1:
            return (sum(books[s:]), ((s, n),))
        cases = (
            recalc(prog(p-1, s2), sum(books[s:s2]), ((s, s2),))
            for s2 in range(s+1, n+1-p))
        return min(cases, key=lambda e: e[0])
    return prog(p, 0)

books = [1, 2, 3, 4, 5, 6, 7, 8, 9]
parts = [[1, 2, 3, 4, 5], [6, 7], [8, 9]]
assert parts == [books[s:e] for s, e in bookshelf(books, 3)[1]]

In [10]:
def make_amount(k, denoms): # denominations, how many ways to make the amount k?
    denoms, d = denoms[:-1], denoms[-1]
    return sum(make_amount(k-d*n, denoms) for n in range(1+k//d)) if d > 1 else 1

def make_changes(k, denoms): # denominations, make the change k with least coins.
    @lru_cache(maxsize=None)
    def prog(k):
        return min(((d,) + prog(k-d) for d in denoms if k >= d), key=len) if k > 0 else ()
    return prog(k)

assert 1 == make_amount(4, [1, 5, 10, 25])
assert 2 == make_amount(5, [1, 5, 10, 25])
assert 4 == make_amount(10, [1, 5, 10, 25]) # 4 ways to make 10 cents
assert 9 == make_amount(20, [1, 5, 10, 25]) # 13 ways to make 25 cents.

assert (5, 5) == make_changes(10, (7, 5, 1))
assert (7, 5, 1) == make_changes(13, (7, 5, 1))
assert (7, 7) == make_changes(14, (7, 5, 1))